In [108]:
    ##Borrar datos del entorno
rm(list=ls())


#Se utiliza pacman para instalar y cargar paquetes
if(!require('pacman')) install.packages('pacman')
pacman::p_load(tidyverse,janitor,httr,jsonlite,gt)


In [109]:
#Función para consultar la api

consulta_api<-function(url){
  r <- GET(url, timeout(10000))
  df <- fromJSON(rawToChar(r$content))
  return(df)
}

# Obtener datos de autoproducción por ONAVI


### SHF

In [110]:
url="https://sniiv.sedatu.gob.mx/api/CuboAPI/GetFinanciamiento/2024,2024/0/0/organismo,modalidad,destino_credito,anio,mes,rango_edad,genero"


In [111]:
finan_gral<-consulta_api(url)%>%
arrange(año)
str(finan_gral)

'data.frame':	323 obs. of  9 variables:
 $ año            : int  2024 2024 2024 2024 2024 2024 2024 2024 2024 2024 ...
 $ mes            : chr  "enero" "enero" "enero" "enero" ...
 $ organismo      : chr  "FOVISSSTE" "FOVISSSTE" "FOVISSSTE" "FOVISSSTE" ...
 $ modalidad      : chr  "Viviendas nuevas" "Viviendas nuevas" "Viviendas nuevas" "Viviendas nuevas" ...
 $ destino_credito: chr  "Vivienda nueva" "Vivienda nueva" "Vivienda nueva" "Vivienda nueva" ...
 $ sexo           : chr  "Hombre" "Mujer" "Hombre" "Mujer" ...
 $ grupo_edad     : chr  "29 o menos" "29 o menos" "30 a 59" "30 a 59" ...
 $ acciones       : int  42 77 367 510 42 24 43 86 408 506 ...
 $ monto          : num  3.70e+07 5.67e+07 3.34e+08 4.39e+08 4.07e+07 ...


In [112]:
shf <- finan_gral %>%
  filter(organismo == "SHF (FONDEO)" &
         destino_credito %in% c("Autoproducción", "Ampliación", "Reconstrucción", "Mejoramientos"))%>%
group_by(año,organismo, grupo_edad,sexo,modalidad)%>%
summarise(creditos=sum(acciones),monto=sum(monto))
head(shf,5)

`summarise()` has grouped output by 'año', 'organismo', 'grupo_edad', 'sexo'.
You can override using the `.groups` argument.


año,organismo,grupo_edad,sexo,modalidad,creditos,monto
<int>,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>
2024,SHF (FONDEO),29 o menos,Hombre,Mejoramientos,3668,0
2024,SHF (FONDEO),29 o menos,Mujer,Mejoramientos,7050,0
2024,SHF (FONDEO),No disponible,No disponible,Mejoramientos,0,516392359


### Fovissste

In [113]:
fovissste<-finan_gral%>%
filter(organismo=="FOVISSSTE" &
       destino_credito %in% c("Con disponibilidad de terreno"))%>%
group_by(año,organismo, grupo_edad,sexo,modalidad)%>%
summarise(creditos=sum(acciones),monto=sum(monto))
head(fovissste,5)

`summarise()` has grouped output by 'año', 'organismo', 'grupo_edad', 'sexo'.
You can override using the `.groups` argument.


año,organismo,grupo_edad,sexo,modalidad,creditos,monto
<int>,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>
2024,FOVISSSTE,29 o menos,Hombre,Viviendas nuevas,2,1372342
2024,FOVISSSTE,29 o menos,Mujer,Viviendas nuevas,1,1377500
2024,FOVISSSTE,30 a 59,Hombre,Viviendas nuevas,12,12336102
2024,FOVISSSTE,30 a 59,Mujer,Viviendas nuevas,39,35481642
2024,FOVISSSTE,60 o más,Hombre,Viviendas nuevas,1,2095448


### Infonavit   

In [114]:
url_infonavit<-"https://qa-sniiv.sedatu.gob.mx/api/CuboAPI/GetINFONAVIT/2024,2024/0/0/anio,linea_credito,modalidad,genero,rango_edad"

In [115]:
bdinfonavit<-consulta_api(url_infonavit)%>%
arrange(año)

In [116]:
infonavit<-bdinfonavit%>%
filter(linea_credito %in% c("Linea II", "Linea III","Linea IV"))%>%
filter(modalidad %in% c("Adquisicion de suelo", "Ampliación y rehabilitación","Autoconstrucción","Con disponibilidad de terreno"))%>%
group_by(año,grupo_edad,sexo,modalidad)%>%
summarise(creditos=sum(acciones),monto=sum(monto))%>%
mutate(organismo="INFONAVIT")
head(infonavit,5)

`summarise()` has grouped output by 'año', 'grupo_edad', 'sexo'. You can
override using the `.groups` argument.


año,grupo_edad,sexo,modalidad,creditos,monto,organismo
<int>,<chr>,<chr>,<chr>,<int>,<dbl>,<chr>
2024,29 o menos,Hombre,Ampliación y rehabilitación,4533,11089842,INFONAVIT
2024,29 o menos,Hombre,Autoconstrucción,2,0,INFONAVIT
2024,29 o menos,Hombre,Con disponibilidad de terreno,672,400450606,INFONAVIT
2024,29 o menos,Mujer,Ampliación y rehabilitación,1733,7947528,INFONAVIT
2024,29 o menos,Mujer,Autoconstrucción,4,0,INFONAVIT


### Conavi

In [117]:
url_conavi<-"https://qa-sniiv.sedatu.gob.mx/api/CuboAPI/GetCONAVI/2024,2024/0/0/modalidad,anio,mes,genero,rango_edad"

In [118]:
bdconavi<-consulta_api(url_conavi)%>%
filter(mes!="abril")%>%
arrange(año)

In [119]:
conavi<-bdconavi%>%
filter(modalidad %in% c("Autoproducción", "Autoconstrucción", "Ampliación", "Mejoramiento","Reconstrucción","Vivienda existente"))%>%
group_by(año,grupo_edad,sexo,modalidad)%>%

summarise(creditos=sum(acciones),monto=sum(monto))%>%
mutate(organismo="CONAVI")


conavi

`summarise()` has grouped output by 'año', 'grupo_edad', 'sexo'. You can
override using the `.groups` argument.


año,grupo_edad,sexo,modalidad,creditos,monto,organismo
<int>,<chr>,<chr>,<chr>,<int>,<dbl>,<chr>
2024,29 o menos,Hombre,Mejoramiento,2610,91350000,CONAVI
2024,29 o menos,Mujer,Mejoramiento,6159,215565000,CONAVI
2024,30 a 59,Hombre,Mejoramiento,15628,546980000,CONAVI
2024,30 a 59,Mujer,Mejoramiento,27627,966945000,CONAVI
2024,60 o más,Hombre,Mejoramiento,9116,319060000,CONAVI
2024,60 o más,Mujer,Mejoramiento,9480,331800000,CONAVI


In [122]:
autoproduccion<-bind_rows(shf,fovissste,infonavit,conavi)
print(sprintf("Total de financiamientos para autoproducción: %s", format(sum(autoproduccion$creditos), big.mark = ",")))
#Guardar datos en excel
openxlsx::write.xlsx(autoproduccion,"autoproduccion_1T_2024.xlsx")

[1] "Total de financiamientos para autoproducción: 134,181"
